In [1]:
import numpy as np
import tensorflow as tf

In [2]:
# !pip install tensorflow_addons


In [3]:
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D,GRU
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, Concatenate
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.optimizers import Adam
import tensorflow_addons as tfa
import json
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd


C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [11]:
max_len=57
feat_vec=300

In [12]:
with open ("../../encode_data_shopee.txt", "r") as f:
    encode_data_shopee = json.loads(f.read())
with open ("../../encode_data_tiki.txt", "r") as f:
    encode_data_tiki = json.loads(f.read())

In [13]:
max_len = 0
for i in range(len(encode_data_shopee)):
     if len(encode_data_shopee[i]) > max_len:
            max_len = len(encode_data_shopee[i])
max_len

57

In [14]:
embed_mat = np.load("./weights/embedding_mat.npy")
embed_mat.shape

(5237, 300)

In [15]:
padded_doc_shopee=pad_sequences(encode_data_shopee,maxlen=max_len,padding="post")
padded_doc_tiki=pad_sequences(encode_data_tiki,maxlen=max_len,padding="post")


In [16]:
with open ("../../output_shopee.txt", "r") as f:
    output_ori_shopee = np.array(json.loads(f.read()))
with open ("../../output_tiki.txt", "r") as f:
    output_ori_tiki = np.array(json.loads(f.read()))

In [17]:
print(padded_doc_shopee.shape, output_ori_shopee.shape)
X_train_shopee, X_test_shopee, y_train_ori_shopee, y_test_ori_shopee = train_test_split(padded_doc_shopee, output_ori_shopee, test_size=0.2, random_state=14)

(2951, 57) (2951, 7)


In [18]:
y_train_shopee = np.delete(y_train_ori_shopee, 6, 1)
y_test_shopee = np.delete(y_test_ori_shopee, 6, 1)
# output_train_shopee = np.delete(output_ori_shopee, 6, 1)

In [19]:
print(np.__version__)

1.18.5


In [20]:
f1 = tfa.metrics.F1Score(num_classes=6, average='macro', threshold=0.5)
# rlp = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',mode='min', patience=3, verbose=1, factor=0.4)
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss',mode='min', patience=6, verbose=1, restore_best_weights=True)
filepath = ('./weights/best_teacher.h5')
checkpoint = ModelCheckpoint(filepath, monitor='val_f1_score', verbose=1, save_best_only=True,save_weights_only=True, mode='max')

In [17]:
# !pip install -U --user numpy==1.18.5
# model = get_model(padded_doc, X_test, output_ori_pre, y_test)

In [ ]:
def get_model():
    inp = Input(shape=(max_len,))
    x = Embedding(len(embed_mat), feat_vec, weights=[np.array(embed_mat)], input_length=max_len,trainable=True)(inp) # trọng số của từ sẽ không được train lại
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Conv1D(64,3,activation="relu")(x)
    x = GlobalMaxPool1D()(x)
    x = Dense(256, activation="relu")(x)
#     convo1 = Conv1D(64,3,activation="relu")(x)
#     convo1_maxpool = GlobalMaxPool1D()(convo1)
#     convo2 = Conv1D(128,4,activation="relu")(x)
#     convo2_maxpool = GlobalMaxPool1D()(convo2)
#     convo3 = Conv1D(64,5,activation="relu")(x)
#     convo3_maxpool = GlobalMaxPool1D()(convo3)
#     concat1 = Concatenate()([convo1_maxpool, convo2_maxpool])
#     concat2 = Concatenate()([concat1, convo3_maxpool])
#     x = Dense(256, activation="relu")(concat2)
    
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    print(model.summary())

    return model
    
# model = get_model(X_train, X_test, y_train, y_test)
# model = get_model(padded_doc_shopee, X_test_tiki, output_train_shopee, y_test_tiki)
model = get_model()
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.005), metrics= [f1])
model.fit(X_train_shopee, y_train_shopee, batch_size=64, epochs=30, validation_data=(X_test_shopee, y_test_shopee), callbacks=[checkpoint])

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 57)]              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 57, 300)           1571100   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 57, 256)           439296    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 55, 64)            49216     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               16640     
_________________________________________________________________
dense_5 (Dense)              (None, 6)                

In [ ]:
# print(model.summary())

In [ ]:
names = [weight.name for layer in model.layers for weight in layer.weights]
weights = model.get_weights()

for name, weight in zip(names, weights):
    print(name, weight.shape)

In [22]:
new_model = get_model()
new_model.load_weights('./weights/best_teacher.h5')

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 57)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 57, 300)           1571100   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 57, 256)           439296    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 55, 64)            49216     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               16640     
_________________________________________________________________
dense_3 (Dense)              (None, 6)                

In [29]:
model.layers[0].name
weight_1=new_model.layers[1].get_weights()
print(weight_1[0].shape)
# with open('./weights/embedding_trained_mat.txt', 'w') as f:
#     f.write(json.dumps(weight_1[0].tolist()))
np.save('./weights/embedding_trained_mat', weight_1)

(5237, 300)


In [30]:
weight_2 = new_model.layers[2].get_weights()
len(weight_2[0])
# with open('./weights/lstm_trained_mat.txt', 'w') as f:
#     f.write(json.dumps(np.array(weight_2).tolist()))
np.save('./weights/lstm_trained_mat', weight_2)

In [31]:
weight_3 = new_model.layers[3].get_weights()
len(weight_2[0])
# with open('./weights/lstm_trained_mat.txt', 'w') as f:
#     f.write(json.dumps(np.array(weight_2).tolist()))
np.save('./weights/convo_trained_mat', weight_3)

In [23]:
from sklearn import metrics
def evaluate(X_test, y_test, model):    
    y_pre = model.predict(X_test)
    for thresh in np.arange(0.2,0.7,0.01):
        print("threshold {0:2.2f} f1 score:{1:2.3f}".format(thresh,metrics.f1_score(y_test,(y_pre>thresh).astype(int), average='macro')))
    return y_pre

In [24]:
y_pre=evaluate(X_test_shopee, y_test_shopee, new_model)

threshold 0.20 f1 score:0.804
threshold 0.21 f1 score:0.807
threshold 0.22 f1 score:0.807
threshold 0.23 f1 score:0.809
threshold 0.24 f1 score:0.809
threshold 0.25 f1 score:0.812
threshold 0.26 f1 score:0.812
threshold 0.27 f1 score:0.812
threshold 0.28 f1 score:0.814
threshold 0.29 f1 score:0.815
threshold 0.30 f1 score:0.814
threshold 0.31 f1 score:0.815
threshold 0.32 f1 score:0.814
threshold 0.33 f1 score:0.816
threshold 0.34 f1 score:0.814
threshold 0.35 f1 score:0.816
threshold 0.36 f1 score:0.816
threshold 0.37 f1 score:0.818
threshold 0.38 f1 score:0.818
threshold 0.39 f1 score:0.819
threshold 0.40 f1 score:0.819
threshold 0.41 f1 score:0.819
threshold 0.42 f1 score:0.819
threshold 0.43 f1 score:0.818
threshold 0.44 f1 score:0.817
threshold 0.45 f1 score:0.817
threshold 0.46 f1 score:0.817
threshold 0.47 f1 score:0.816
threshold 0.48 f1 score:0.815
threshold 0.49 f1 score:0.815
threshold 0.50 f1 score:0.813
threshold 0.51 f1 score:0.812
threshold 0.52 f1 score:0.811
threshold 

In [25]:
thresh=0.41
y_pre_round = []
for i in range(len(y_pre)):
    y_pre_round.append(1*(y_pre[i] >= thresh))
    y_pre_round[i] = y_pre_round[i].tolist()
    if 1 not in y_pre_round[i]:
        y_pre_round[i].append(1)
    else: y_pre_round[i].append(0)
 

In [26]:
import sys
sys.path.append(r"C:\Users\acer\Documents\hoc tap\khóa luận\mebe_domain")


In [27]:
from modules.evaluate import cal_aspect_prf

X = cal_aspect_prf(y_test_ori_shopee.tolist(), y_pre_round, num_of_aspect=7, verbal=True)
# print(X)

p: [0.9458762886597938, 0.9021739130434783, 0.75, 0.8043478260869565, 0.7638888888888888, 0.8764044943820225, 0.3048780487804878]
r: [0.9458762886597938, 0.9431818181818182, 0.72, 0.7254901960784313, 0.6626506024096386, 0.8041237113402062, 0.5952380952380952]
f1: [0.9458762886597938, 0.9222222222222224, 0.7346938775510204, 0.7628865979381443, 0.7096774193548386, 0.8387096774193549, 0.40322580645161293]
micro: (0.875139353400223, 0.839572192513369, 0.8569868995633189)
macro: (0.84044856851019, 0.8002204361116481, 0.8190110138575624)


In [42]:
# label = ['ship','giá','chính hãng''','chất lượng','dịch vụ','an toàn','others']
# res = {'label':label,'p':X[0], 'r':X[1], 'f1':X[2]}
# df = pd.DataFrame(res)
# df1 = pd.DataFrame({'micro':X[3], 'macro': X[4]})
# final_df=pd.concat([df,df1], axis=1)

# final_df.to_csv(r"C:\Users\acer\Documents\hoc tap\khóa luận\mebe_domain_pre\ket qua\baseline.csv")